In [1]:
# The code was removed by Watson Studio for sharing.

In [22]:
#Data Extracted from The COVID Tracking Project (as of 4/8/2020)
#https://covidtracking.com/
import pandas as pd
import numpy as np
import requests
import json
import datetime
!pip install pycountry
import pycountry
!pip install 'wget'
import wget
import datetime
!pip install 'ibmos2spark'
import ibmos2spark

In [3]:
raw_response = requests.get("https://covidtracking.com/api/states/daily").text
raw_data = pd.DataFrame.from_dict(json.loads(raw_response))

In [4]:
data = raw_data.fillna(0)
data['Date'] = pd.to_datetime(data['date'].astype(str), format='%Y%m%d')
data = data.rename(
    columns={
        "state": "ISO3166-2",
        "positive": "Positive",
        "negative": "Negative",
        "pending": "Pending",
        "death": "Death",
        "totalTestResults": "Total",
        "hospitalized": "Hospitalized"
    })
data = data.drop(labels=['dateChecked', "date"], axis='columns')
data['Country/Region'] = "United States"
data['ISO3166-1'] = "US"

In [5]:
states = {k.code.replace("US-", ""): k.name for k in pycountry.subdivisions.get(country_code="US")}

In [6]:
data["Province/State"] = data["ISO3166-2"].apply(lambda x: states[x])

In [7]:
sorted_data = data.sort_values(by=['Province/State'] + ['Date'], ascending=True)
sorted_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6513 entries, 6341 to 55
Data columns (total 41 columns):
checkTimeEt                 6513 non-null object
commercialScore             6513 non-null int64
dataQualityGrade            6513 non-null object
dateModified                6513 non-null object
Death                       6513 non-null float64
deathIncrease               6513 non-null int64
fips                        6513 non-null object
grade                       6513 non-null object
hash                        6513 non-null object
Hospitalized                6513 non-null float64
hospitalizedCumulative      6513 non-null float64
hospitalizedCurrently       6513 non-null float64
hospitalizedIncrease        6513 non-null int64
inIcuCumulative             6513 non-null float64
inIcuCurrently              6513 non-null float64
lastUpdateEt                6513 non-null object
Negative                    6513 non-null float64
negativeIncrease            6513 non-null int64
negativ

In [51]:
#Hospitalized and hospitalizedCumulative both appear to be carrying cumulative numbers for MA
#Positive, Negative & Total are patient counts.  totalTestsViral is all tests (repeats on same patient).  
#The Total appears to include a patient twice if they have both a positive and a negative test its not an exact 
#match for the MA DPH unique patients tested figure.  Other states are not reporting into totalTestsViral so Total is still being used.
ma_test = sorted_data[(sorted_data['Province/State']=='Kentucky')]
ma_test.filter(items=['Date','Hospitalized', 'hospitalizedCumulative', 'hospitalizedCurrently']).tail()
ma_test.filter(items=['Date','Province/State','Total', 'total','totalTestsViral', 'Positive', 'Negative', 'Pending', 'Death']).tail()

,Date,Province/State,Total,total,totalTestsViral,Positive,Negative,Pending,Death
243,2020-06-25,Kentucky,342882,342882,0.0,14617.0,328265.0,0.0,546.0
187,2020-06-26,Kentucky,350754,350754,0.0,14859.0,335895.0,0.0,553.0
131,2020-06-27,Kentucky,350754,350754,0.0,14859.0,335895.0,0.0,553.0
75,2020-06-28,Kentucky,358491,358491,0.0,15232.0,343259.0,0.0,558.0
19,2020-06-29,Kentucky,361441,361441,0.0,15347.0,346094.0,0.0,560.0


In [8]:
sorted_data['Current_Day_Positive_Cases'] = sorted_data['Positive'] - sorted_data.groupby(['Province/State'])["Positive"].shift(1, fill_value=0)
sorted_data['Current_Day_Total_Tests'] = sorted_data['Total'] - sorted_data.groupby(['Province/State'])["Total"].shift(1, fill_value=0)
sorted_data['Current_Day_Negative_Cases'] = sorted_data['Negative'] - sorted_data.groupby(['Province/State'])["Negative"].shift(1, fill_value=0)
sorted_data['Current_Day_Pending_Cases'] = sorted_data['Pending'] - sorted_data.groupby(['Province/State'])["Pending"].shift(1, fill_value=0)
sorted_data['Current_Day_Deaths'] = sorted_data['Death'] - sorted_data.groupby(['Province/State'])["Death"].shift(1, fill_value=0)
sorted_data['Current_Day_New_Hospitalized'] = sorted_data['Hospitalized'] - sorted_data.groupby(['Province/State'])["Hospitalized"].shift(1, fill_value=0)
sorted_data['Currently_Hospitalized'] = sorted_data['hospitalizedCurrently'] 
sorted_data['Prior_Day_Positive_Cases'] = sorted_data.groupby('Province/State')['Positive'].shift(1, fill_value=0)
sorted_data['Prior_Day_Deaths'] = sorted_data.groupby('Province/State')['Death'].shift(1, fill_value=0)
sorted_data['Prior_Day_Total_Tests'] = sorted_data.groupby('Province/State')['Total'].shift(1, fill_value=0)
sorted_data['Pct_Change_Positive_Cases'] = (sorted_data['Positive'] - sorted_data['Prior_Day_Positive_Cases'])/sorted_data['Prior_Day_Positive_Cases']
sorted_data['Pct_Change_Deaths'] = (sorted_data['Death'] - sorted_data['Prior_Day_Deaths'])/sorted_data['Prior_Day_Deaths']
sorted_data['Pct_Change_Total_Tests'] = (sorted_data['Total'] - sorted_data['Prior_Day_Total_Tests'])/sorted_data['Prior_Day_Total_Tests']
sorted_data['Cumulative_Positive_Cases'] = sorted_data['Positive']
sorted_data['Cumulative_Total_Tests'] = sorted_data['Total']
sorted_data['Cumulative_Deaths'] = sorted_data['Death']
sorted_data['Cumulative_Hospitalized'] = sorted_data['hospitalizedCumulative']

In [9]:
#bring in csv for state population estimates
urls = ['https://raw.githubusercontent.com/rdewolf127/DataVisualization611/master/state_population_est_2019.csv']
for url in urls:
    filename = wget.download(url)
filename

'state_population_est_2019.csv'

In [10]:
state_pop = pd.read_csv('state_population_est_2019.csv', dtype={'fips':'str'})
state_pop.head()

,fips,Province/State,PopEst_2019,Per_500K_Factor
0,01,.Alabama,"4,903,185",9.806370
1,02,.Alaska,"731,545",1.463090
2,04,.Arizona,"7,278,717",14.557434
3,05,.Arkansas,"3,017,804",6.035608
4,06,.California,"39,512,223",79.024446


In [11]:
#bring in csv for state population estimates
urls = ['https://raw.githubusercontent.com/rdewolf127/DataVisualization611/master/state_population_density.csv']
for url in urls:
    filename = wget.download(url)
filename

'state_population_density.csv'

In [12]:
state_density = pd.read_csv('state_population_density.csv', dtype={'fips':'str'})
state_density.head()

,fips,state,landareasquaremiles,popdensity19
0,01,Alabama,50645.33,96.81
1,02,Alaska,553559.49,1.32
2,04,Arizona,113594.09,64.08
3,05,Arkansas,52035.44,58.00
4,06,California,155779.19,253.64


In [13]:
merged_data = pd.merge(sorted_data, state_pop, on='fips')
merged_data = pd.merge(merged_data, state_density, on='fips')
merged_data['Province/State']=merged_data['Province/State_x']
merged_data.head()

,checkTimeEt,commercialScore,dataQualityGrade,dateModified,Death,deathIncrease,fips,grade,hash,Hospitalized,...,Cumulative_Total_Tests,Cumulative_Deaths,Cumulative_Hospitalized,Province/State_y,PopEst_2019,Per_500K_Factor,state,landareasquaremiles,popdensity19,Province/State
0,0,0,,0,0.0,0,01,,107d5796e64511acf181941412d9aa91f4d75ad2,0.0,...,0,0.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama
1,0,0,,0,0.0,0,01,,7facc09db7bcdfed34ef772657473471943c745b,0.0,...,0,0.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama
2,0,0,,0,0.0,0,01,,b8d68b71b5d9fffdfdb973e9198bb2b73438fd3f,0.0,...,0,0.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama
3,0,0,,0,0.0,0,01,,631b4d0aee9e1e09bcc57a2b79fda9d33899925f,0.0,...,0,0.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama
4,0,0,,0,0.0,0,01,,a72d7525867debf1c3825ce9bcc1e6b805499fc9,0.0,...,10,0.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama


In [14]:
rearranged_data = merged_data.filter(items=['fips','Country/Region', 'Province/State', 'Date',
                               'Cumulative_Positive_Cases', 'Current_Day_Positive_Cases',
                               'Prior_Day_Positive_Cases','Pct_Change_Positive_Cases',
                               'Cumulative_Total_Tests', 'Current_Day_Total_Tests',
                               'Prior_Day_Total_Tests', 'Pct_Change_Total_Tests',
                               'Cumulative_Deaths', 'Current_Day_Deaths',
                               'Prior_Day_Deaths', 'Pct_Change_Deaths',
                               'Current_Day_New_Hospitalized','Cumulative_Hospitalized', 'Currently_Hospitalized',
                               'ISO3166-1', 'ISO3166-2', 'PopEst_2019', 'Per_500K_Factor','landareasquaremiles', 'popdensity19'])

In [15]:
rearranged_data.loc[:, "Last_Update_Date"] = datetime.datetime.utcnow()
rearranged_data.head()

,fips,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,...,Current_Day_New_Hospitalized,Cumulative_Hospitalized,Currently_Hospitalized,ISO3166-1,ISO3166-2,PopEst_2019,Per_500K_Factor,landareasquaremiles,popdensity19,Last_Update_Date
0,01,United States,Alabama,2020-03-07,0.0,0.0,0.0,NaN,0,0,...,0.0,0.0,0.0,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-06-30 13:47:59.056560
1,01,United States,Alabama,2020-03-08,0.0,0.0,0.0,NaN,0,0,...,0.0,0.0,0.0,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-06-30 13:47:59.056560
2,01,United States,Alabama,2020-03-09,0.0,0.0,0.0,NaN,0,0,...,0.0,0.0,0.0,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-06-30 13:47:59.056560
3,01,United States,Alabama,2020-03-10,0.0,0.0,0.0,NaN,0,0,...,0.0,0.0,0.0,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-06-30 13:47:59.056560
4,01,United States,Alabama,2020-03-11,0.0,0.0,0.0,NaN,10,10,...,0.0,0.0,0.0,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-06-30 13:47:59.056560


In [16]:
rearranged_data['Cumulative_Mortality_Rate']=rearranged_data['Cumulative_Deaths']/rearranged_data['Cumulative_Positive_Cases']
rearranged_data['Cumulative_Positive_Test_Rate'] = rearranged_data['Cumulative_Positive_Cases']/rearranged_data['Cumulative_Total_Tests']
rearranged_data['Current_Day_Mortality_Rate']=rearranged_data['Current_Day_Deaths']/rearranged_data['Current_Day_Positive_Cases']
rearranged_data['Current_Day_Positive_Test_Rate'] = rearranged_data['Current_Day_Positive_Cases']/rearranged_data['Current_Day_Total_Tests']
rearranged_data['Cumulative_Positive_Cases_Per_500K'] = round(rearranged_data['Cumulative_Positive_Cases']/rearranged_data['Per_500K_Factor'],0)
rearranged_data['Cumulative_Positive_Cases_Per_500K - LOG'] = np.log(rearranged_data['Cumulative_Positive_Cases_Per_500K'])
rearranged_data['popdensity19 - LOG'] = np.log(rearranged_data['popdensity19'])
rearranged_data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in log


,fips,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,...,landareasquaremiles,popdensity19,Last_Update_Date,Cumulative_Mortality_Rate,Cumulative_Positive_Test_Rate,Current_Day_Mortality_Rate,Current_Day_Positive_Test_Rate,Cumulative_Positive_Cases_Per_500K,Cumulative_Positive_Cases_Per_500K - LOG,popdensity19 - LOG
0,01,United States,Alabama,2020-03-07,0.0,0.0,0.0,NaN,0,0,...,50645.33,96.81,2020-06-30 13:47:59.056560,NaN,NaN,NaN,NaN,0.0,-inf,4.57275
1,01,United States,Alabama,2020-03-08,0.0,0.0,0.0,NaN,0,0,...,50645.33,96.81,2020-06-30 13:47:59.056560,NaN,NaN,NaN,NaN,0.0,-inf,4.57275
2,01,United States,Alabama,2020-03-09,0.0,0.0,0.0,NaN,0,0,...,50645.33,96.81,2020-06-30 13:47:59.056560,NaN,NaN,NaN,NaN,0.0,-inf,4.57275
3,01,United States,Alabama,2020-03-10,0.0,0.0,0.0,NaN,0,0,...,50645.33,96.81,2020-06-30 13:47:59.056560,NaN,NaN,NaN,NaN,0.0,-inf,4.57275
4,01,United States,Alabama,2020-03-11,0.0,0.0,0.0,NaN,10,10,...,50645.33,96.81,2020-06-30 13:47:59.056560,NaN,0.0,NaN,0.0,0.0,-inf,4.57275


In [17]:
rearranged_data['MA7_Pct_Change_Positive_Cases'] = rearranged_data.rolling(window=7)['Pct_Change_Positive_Cases'].mean()
rearranged_data['MA7_Pct_Change_Total_Tests'] = rearranged_data.rolling(window=7)['Pct_Change_Total_Tests'].mean()
rearranged_data['MA7_Pct_Change_Deaths'] = rearranged_data.rolling(window=7)['Pct_Change_Deaths'].mean()
rearranged_data['MA7_Cumulative_Mortality_Rate'] = rearranged_data.rolling(window=7)['Cumulative_Mortality_Rate'].mean()
rearranged_data['MA7_Current_Day_Mortality_Rate'] = rearranged_data.rolling(window=7)['Current_Day_Mortality_Rate'].mean()
rearranged_data['MA7_Cumulative_Positive_Test_Rate'] = rearranged_data.rolling(window=7)['Cumulative_Positive_Test_Rate'].mean()
rearranged_data['MA7_Current_Day_Positive_Test_Rate'] = rearranged_data.rolling(window=7)['Current_Day_Positive_Test_Rate'].mean()
rearranged_data['MA7_Current_Day_Positive_Cases'] = rearranged_data.rolling(window=7)['Current_Day_Positive_Cases'].mean()
rearranged_data['MA7_Currently_Hospitalized'] = rearranged_data.rolling(window=7)['Currently_Hospitalized'].mean()
rearranged_data.tail()

,fips,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,...,popdensity19 - LOG,MA7_Pct_Change_Positive_Cases,MA7_Pct_Change_Total_Tests,MA7_Pct_Change_Deaths,MA7_Cumulative_Mortality_Rate,MA7_Current_Day_Mortality_Rate,MA7_Cumulative_Positive_Test_Rate,MA7_Current_Day_Positive_Test_Rate,MA7_Current_Day_Positive_Cases,MA7_Currently_Hospitalized
5978,56,United States,Wyoming,2020-06-25,1326.0,47.0,1279.0,0.036747,30195,47,...,1.78507,0.021357,-0.019989,1.587302e-02,0.016236,0.009852,0.041987,0.446504,26.000000,7.714286
5979,56,United States,Wyoming,2020-06-26,1368.0,42.0,1326.0,0.031674,31774,1579,...,1.78507,0.022260,-0.013783,3.628050e-16,0.015889,0.000000,0.043543,0.437358,27.857143,7.285714
5980,56,United States,Wyoming,2020-06-27,1392.0,24.0,1368.0,0.017544,31798,24,...,1.78507,0.024036,0.022096,3.628050e-16,0.015518,0.000000,0.043637,0.580309,30.428571,6.857143
5981,56,United States,Wyoming,2020-06-28,1417.0,25.0,1392.0,0.017960,31823,25,...,1.78507,0.024420,0.022115,3.628050e-16,0.015147,0.000000,0.043748,0.580309,31.428571,6.428571
5982,56,United States,Wyoming,2020-06-29,1450.0,33.0,1417.0,0.023289,31856,33,...,1.78507,0.023809,0.022090,3.628050e-16,0.014795,0.000000,0.043835,0.580309,31.428571,6.285714


In [18]:
MA_only = rearranged_data[(rearranged_data['Province/State']=='Massachusetts')]
MA_only.head()

,fips,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,...,popdensity19 - LOG,MA7_Pct_Change_Positive_Cases,MA7_Pct_Change_Total_Tests,MA7_Pct_Change_Deaths,MA7_Cumulative_Mortality_Rate,MA7_Current_Day_Mortality_Rate,MA7_Cumulative_Positive_Test_Rate,MA7_Current_Day_Positive_Test_Rate,MA7_Current_Day_Positive_Cases,MA7_Currently_Hospitalized
2443,25,United States,Massachusetts,2020-03-12,8.0,8.0,0.0,inf,8,8,...,6.784061,NaN,NaN,NaN,0.040689,0.032538,0.254363,0.185186,322.142857,416.142857
2444,25,United States,Massachusetts,2020-03-13,23.0,15.0,8.0,1.875000,23,15,...,6.784061,NaN,NaN,NaN,0.033893,0.025611,0.378189,0.321233,277.142857,338.428571
2445,25,United States,Massachusetts,2020-03-14,38.0,15.0,23.0,0.652174,38,15,...,6.784061,NaN,NaN,NaN,0.027098,0.018793,0.502213,0.456697,216.428571,265.428571
2446,25,United States,Massachusetts,2020-03-15,64.0,26.0,38.0,0.684211,648,610,...,6.784061,NaN,NaN,NaN,0.020309,0.013299,0.497671,0.456294,171.857143,195.857143
2447,25,United States,Massachusetts,2020-03-16,97.0,33.0,64.0,0.515625,1296,648,...,6.784061,NaN,NaN,NaN,0.013522,0.006902,0.489888,0.457118,128.714286,127.571429


In [19]:
date=datetime.date.today().isoformat()
date
date_csv_all = 'all_us_'+date+'.csv'
date_csv_all
date_csv_ma = 'ma_only_'+date+'.csv'
date_csv_ma

'ma_only_2020-06-30.csv'

In [23]:
# The code was removed by Watson Studio for sharing.

2020-06-30 13:50:39,549 - __PROJECT_LIB__ - ERROR - failed to initialize ibmos2spark integration
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/project_lib/storage/bcos.py", line 141, in _initialize_bcos2spark
    spark_context, credentials, config_name, "bluemix_cos")
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibmos2spark/osconfig.py", line 226, in __init__
    hconf = sparkcontext._jsc.hadoopConfiguration()
AttributeError: 'str' object has no attribute '_jsc'


{'file_name': 'all_us_2020-06-30.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'covid19-donotdelete-pr-ysdyv5czxoifrj',
 'asset_id': 'd8458c8c-dd00-48b5-b5cf-0b3757f20064'}